# 0 — Problema, Riesgo y Decisiones (antes del modelo)

> **De métricas a decisiones. De modelos a sistemas gobernados.**  
> En este notebook no “entrenamos un modelo”. Definimos el **problema real**: la **decisión** que el sistema va a tomar y el **riesgo operativo** asociado.
---

## 1) Propósito del notebook

Este notebook establece el marco de trabajo de toda la serie:

- Aterrizar **qué decisión** queremos automatizar (y cuál no).
- Traducir el problema en **riesgo**, **costos de error** y **políticas de acción**.
- Definir el **contrato de decisión**: qué significa “aceptar”, “rechazar” o “revisar”.
- Preparar el terreno para que, más adelante, métricas, calibración, thresholds e incertidumbre (CeRTS) tengan **sentido operativo**.

## 2) Qué vamos a construir aquí (sin modelos)

Vamos a producir tres entregables concretos:

### A. Mapa de decisión
Una especificación simple y accionable:

- **Decisión**: qué está decidiendo el sistema.
- **Acción**: qué pasa cuando decide.
- **Actores**: quién asume el impacto (cliente, equipo, negocio).
- **Restricciones**: lo que no se puede permitir (compliance, fraude, reputación, etc.).

### B. Matriz de costos (sin datos)
Definiremos el costo/impacto relativo de los errores:

- **Falso positivo (FP)**: el sistema actúa cuando no debía.
- **Falso negativo (FN)**: el sistema no actúa cuando debía.
- **Impacto**: financiero, operativo, reputacional, regulatorio.

Esto se traduce en una **preferencia explícita**: qué error “duele más”.

### C. Política de decisión (pre-modelo)
Diseñaremos una política operacional con tres estados:

- **Auto-approve** (automatizar)
- **Auto-reject** (automatizar)
- **Human review** (escalar)

Aquí dejamos claro un principio que gobernará toda la serie:

> **El modelo produce señales. La política decide.**

## 3) Cómo lo haremos (metodología)

Trabajaremos como si estuviéramos definiendo un producto de ML con gobernanza:

1. **Caso de uso**: describimos el escenario y el resultado deseado.
2. **Decisión + acción**: definimos qué se automatiza y qué gatilla esa automatización.
3. **Errores y costos**: priorizamos riesgos, no métricas.
4. **Política**: diseñamos reglas de decisión (incluyendo “no decidir”).
5. **Criterios de éxito**: definimos cómo se evaluará el sistema en términos de negocio.

## 4) Resultados esperados al finalizar

Al terminar este notebook, vas a tener:

- Un **Decision Spec** claro (qué se decide y por qué).
- Una **Risk/Cost Spec** (qué error es inaceptable y qué trade-off aceptas).
- Una **Decision Policy v0** lista para conectarse con:
  - métricas (accuracy, confusion matrix, precision/recall),
  - thresholds como política,
  - calibración de probabilidades,
  - y manejo de incertidumbre con **CeRTS** en notebooks posteriores.

## 5) Qué NO haremos aquí

Para mantener foco estratégico:

- No entrenaremos modelos.
- No haremos EDA.
- No hablaremos de accuracy, ROC o PR.
- No optimizaremos nada.

Este notebook es la **gobernanza previa**: el punto donde se define si el sistema será útil o peligroso.

---
---
---

## 1. Contexto del sistema

Antes de hablar de datos, modelos o métricas, es necesario entender **qué tipo de sistema estamos construyendo** y **en qué contexto opera**.

Este proyecto no busca entrenar un modelo aislado, sino **diseñar un sistema de apoyo a decisiones** donde el machine learning es solo uno de los componentes.
---

---

### 1.1 Descripción general del sistema

El sistema que construiremos tiene como objetivo **evaluar y clasificar situaciones del mundo real** para apoyar una decisión operativa concreta.

En términos generales, el sistema:

- recibe información estructurada (features),
- produce señales estadísticas (scores y probabilidades),
- y **apoya una acción posterior** que puede ser automática o supervisada.

El foco no está en “predecir por predecir”, sino en **decidir con responsabilidad**.

### 1.2 Escenario real de uso

El escenario que usaremos como guía conceptual es el siguiente:

> Un sistema debe evaluar un caso y decidir si:
> - actúa automáticamente,
> - solicita revisión humana,
> - o decide no actuar.

Este tipo de escenario aparece en múltiples dominios:
- calidad de productos,
- evaluación de riesgo,
- sistemas de recomendación con impacto,
- automatización parcial de procesos críticos.

Aunque el dataset sea académico, **el marco mental es industrial y operativo**.

### 1.3 Tipo de decisión que el sistema apoyará

La decisión que el sistema apoya **no es continua**, es **discreta**.

Ejemplos conceptuales:
- aprobar / revisar,
- aceptar / rechazar,
- intervenir / no intervenir.

El modelo no ejecuta la acción directamente.  
El sistema **traduce señales del modelo en decisiones**, bajo reglas explícitas.

### 1.4 ¿Qué significa “automatizar” en este contexto?

Automatizar no significa eliminar al humano.

En este proyecto, automatizar significa:

- definir **cuándo el sistema puede decidir solo**,
- definir **cuándo debe frenar**,
- y definir **cuándo escalar a revisión humana**.

La automatización responsable **incluye explícitamente el derecho a no decidir**.

📌 **Idea clave de esta sección**

> No estamos construyendo un modelo.  
> Estamos diseñando un **sistema de decisión** donde el modelo es solo una pieza.

---
---
---

## 2. La decisión que el sistema debe tomar

Una vez definido el contexto del sistema, el siguiente paso es **especificar con precisión la decisión** que el sistema debe apoyar.

Aquí no hablamos de predicciones, ni de probabilidades, ni de modelos.  
Hablamos de **una acción concreta en el mundo real**.

---

### 2.1 Definición clara de la decisión

La decisión que el sistema debe apoyar puede expresarse de forma simple:

> **¿Es seguro y razonable actuar automáticamente sobre este caso?**

En términos operativos, esta decisión se traduce en algo como:

- **Aceptar / Aprobar automáticamente**
- **Enviar a revisión**
- **No decidir (esperar más información)**

Esta formulación es deliberadamente binaria o ternaria.  
Las decisiones reales rara vez son continuas.

### 2.2 Momento en el que ocurre la decisión

La decisión ocurre **después** de que el sistema ha:

- recibido los datos,
- procesado la información,
- generado señales estadísticas.

Pero ocurre **antes** de cualquier acción irreversible.

Este punto temporal es crítico:  
una decisión tomada demasiado pronto o demasiado tarde **cambia completamente el riesgo**.

### 2.3 Actor que ejecuta la acción

Es fundamental separar:

- **quién propone la decisión**, y
- **quién la ejecuta**.

En este sistema:

- el **modelo** produce señales,
- el **sistema** aplica reglas y políticas,
- el **humano** interviene cuando el sistema lo indica.

El modelo **nunca es el actor final**.

### 2.4 Consecuencias inmediatas de la decisión

Cada decisión tiene efectos directos:

- una acción automática ahorra tiempo y costos,
- una revisión humana introduce fricción,
- una decisión incorrecta puede generar impacto negativo.

Estas consecuencias existen **independientemente del modelo**.

Por eso, definirlas ahora es obligatorio:  
más adelante, todas las métricas deberán **responder a estas consecuencias**.

📌 **Idea clave de esta sección**

> Antes de entrenar un modelo,  
> hay que poder describir la decisión en una sola frase clara.

---
---
---

## 3. Acciones y flujos operativos

Definir la decisión no es suficiente.  
Un sistema de ML responsable necesita **flujos operativos explícitos** que conecten la decisión con acciones reales.

Aquí es donde el diseño deja de ser abstracto y se vuelve **operacional**.

---

### 3.1 Acciones automáticas posibles

Una acción automática es aquella que el sistema puede ejecutar **sin intervención humana inmediata**.

Ejemplos conceptuales:
- aceptar un caso,
- aprobar un registro,
- clasificar un ítem como válido.

Estas acciones **solo son aceptables** cuando el sistema tiene suficiente confianza y bajo riesgo asociado.

Automatizar sin esta claridad es una fuente clásica de fallos en producción.

### 3.2 Acciones que requieren validación humana

No todos los casos deben resolverse automáticamente.

Algunos escenarios requieren:
- revisión experta,
- validación manual,
- análisis contextual adicional.

El sistema debe ser capaz de **detectar estos casos** y redirigirlos al flujo humano **de forma explícita**, no como excepción informal.

### 3.3 Flujo decisión → acción → consecuencia

Toda decisión genera una cadena clara:

1. El sistema evalúa el caso.
2. Se toma una decisión (automática o asistida).
3. Se ejecuta una acción.
4. Se produce una consecuencia.

Este flujo debe ser **observable y trazable**.  
Si no puedes describirlo, no puedes gobernarlo.

### 3.4 Casos donde *no decidir* es una opción válida

Una idea central de este proyecto es aceptar que:

> **No decidir también es una decisión.**

Existen situaciones donde:
- la información es ambigua,
- el riesgo es alto,
- la señal es débil o contradictoria.

En estos casos, **frenar** es la acción correcta.

Diseñar sistemas que saben cuándo no actuar es una señal de madurez, no de debilidad.

📌 **Idea clave de esta sección**

> La calidad de un sistema de ML no se mide solo por sus métricas,  
> sino por la claridad de sus flujos de decisión y acción.

---
---
---

## 4. Tipos de error y su significado real

Antes de hablar de métricas, es imprescindible entender **qué significa equivocarse** en este sistema.

Los errores no son abstractos.  
Tienen consecuencias reales, costos distintos y niveles de tolerancia diferentes.

---

### 4.1 ¿Qué es un falso positivo en este sistema?

Un falso positivo ocurre cuando el sistema **aprueba o actúa automáticamente** sobre un caso que **no debía**.

En términos operativos, esto puede implicar:
- aceptar algo incorrecto,
- ejecutar una acción prematura,
- generar un impacto negativo difícil de revertir.

En muchos sistemas, este es el error **más costoso**, aunque no siempre el más frecuente.

### 4.2 ¿Qué es un falso negativo?

Un falso negativo ocurre cuando el sistema **no actúa** o **envía a revisión** un caso que **sí podía resolverse automáticamente**.

Sus consecuencias suelen ser:
- mayor fricción operativa,
- retrasos,
- aumento de costos humanos.

Aunque molesto, este error suele ser **más tolerable** que un falso positivo.

### 4.3 Errores aceptables vs errores inaceptables

No todos los errores pesan igual.

En este proyecto distinguimos entre:
- errores **operativamente aceptables** (costosos pero manejables),
- errores **operativamente inaceptables** (críticos o irreversibles).

Esta clasificación **no la decide el modelo**.  
La decide el contexto del negocio y del sistema.

### 4.4 Asimetría del costo del error

Un principio clave:

> **Los errores no son simétricos.**

Un falso positivo y un falso negativo pueden tener:
- impactos distintos,
- costos diferentes,
- consecuencias a largo plazo desiguales.

Esta asimetría es la razón por la cual:
- accuracy no es suficiente,
- el threshold importa,
- y la incertidumbre debe medirse.

📌 **Idea clave de esta sección**

> Antes de optimizar una métrica,  
> hay que decidir **qué error estamos dispuestos a aceptar**.

---
---
---

## 5. Riesgo operativo

Una vez identificados los tipos de error, el siguiente paso es entender el **riesgo real** que estos errores introducen en el sistema.

El riesgo no es una métrica.  
Es la **exposición del sistema a consecuencias no deseadas**.

---

### 5.1 Impacto del error en el negocio o sistema

Cada error genera impacto en al menos una de estas dimensiones:
- costo económico,
- fricción operativa,
- pérdida de confianza,
- impacto en usuarios finales.

Un modelo puede equivocarse poco,  
pero si se equivoca **en el lugar incorrecto**, el impacto es alto.

### 5.2 Riesgo legal, financiero y reputacional

Algunos errores trascienden lo técnico:

- **Legal**: incumplimientos, sanciones, responsabilidades.
- **Financiero**: pérdidas directas, costos de corrección.
- **Reputacional**: daño a la marca, pérdida de credibilidad.

Estos riesgos **no aparecen en las métricas estándar**,  
pero dominan la toma de decisiones reales.

### 5.3 Por qué minimizar el error promedio no minimiza el riesgo

Optimizar una métrica global (accuracy, F1, etc.) implica asumir que:
- todos los errores pesan igual,
- el contexto es homogéneo.

En la práctica:
- los errores críticos suelen ser pocos,
- pero concentran la mayor parte del riesgo.

Minimizar el error promedio puede **ocultar fallos graves**.

### 5.4 Relación entre riesgo y automatización

Automatizar significa:
- aceptar riesgo a cambio de velocidad y escala.

Cuanto mayor es la automatización:
- menor intervención humana,
- mayor exposición al error sistémico.

Por eso:
- no todo debe automatizarse,
- no todo debe decidirse,
- y no toda predicción debe convertirse en acción.

📌 **Idea clave de esta sección**

> Automatizar sin medir riesgo  
> es escalar el error.

---
---
---

## 6. Por qué aún no hablamos de métricas

Llegados a este punto, resulta tentador introducir métricas y comenzar a “medir”.  
Sin embargo, hacerlo ahora sería **prematuro y conceptualmente incorrecto**.

---

### 6.1 Qué información falta en este punto

Todavía no hemos definido:
- cómo se representará el problema en datos,
- qué variable objetivo se usará,
- qué modelos se entrenarán,
- ni cómo se producirán las predicciones.

Sin estas piezas, **no existe aún nada que medir**.

### 6.2 Por qué la accuracy aquí sería engañosa

La accuracy responde a una pregunta muy específica:

> “¿Con qué frecuencia el modelo acierta?”

Pero aquí la pregunta correcta es otra:

> “¿Qué pasa cuando el sistema se equivoca?”

Antes de entender:
- el costo del error,
- su asimetría,
- y su impacto operativo,

cualquier métrica global puede generar una **falsa sensación de control**.

### 6.3 Evaluar modelos vs evaluar decisiones

Es clave separar dos niveles:

- **Evaluación del modelo**  
  Se centra en desempeño estadístico.
- **Evaluación de la decisión**  
  Se centra en consecuencias reales.

Un modelo puede ser estadísticamente sólido  
y, aun así, inducir **malas decisiones**.

### 6.4 Preguntas que quedan abiertas

Este notebook deja abiertas, de forma deliberada, preguntas como:

- ¿Qué señal producirá el modelo?
- ¿Score o probabilidad?
- ¿Cómo se convertirá una predicción en acción?
- ¿Cuándo el sistema debería abstenerse de decidir?

Estas preguntas **no se responden aquí**.  
Se responderán más adelante, cuando el contexto esté completo.

📌 **Cierre del notebook**

> Antes de medir, hay que entender qué se está decidiendo  
> y qué se pone en riesgo al automatizar.

---
---
---

## 7. Definición del criterio de éxito (pre-métricas)

Antes de hablar de métricas formales, es imprescindible definir **qué significa que el sistema funcione bien** desde un punto de vista operativo y de negocio.

Este criterio de éxito **no es numérico todavía**. Es conceptual, explícito y verificable.

---

### 7.1 Qué significa “una buena decisión”

Una buena decisión **no es simplemente acertar**.

En este sistema, una buena decisión es aquella que:
- es coherente con el riesgo asumido,
- se toma con información suficiente,
- y produce una consecuencia aceptable incluso cuando falla.

### 7.2 Cuándo el sistema debería frenar

El sistema **no está obligado a decidir siempre**.

Debe ser capaz de:
- detener la automatización,
- escalar a revisión humana,
- o marcar una situación como ambigua,

cuando las condiciones no son adecuadas para una decisión responsable.

### 7.3 Condiciones mínimas que debe cumplir una predicción

Aunque aún no definimos métricas, sí podemos establecer principios como:
- la predicción debe ser consistente,
- no contradictoria con señales alternativas,
- y suficientemente clara para justificar una acción.

Estas condiciones luego se traducirán en **criterios cuantificables**.

### 7.4 Qué se considerará un fallo del sistema

Un fallo del sistema no es solo “equivocarse”.

Es un fallo cuando:
- automatiza una decisión que no debía automatizar,
- ignora señales de incertidumbre,
- o ejecuta una acción sin comprender el impacto.

📌 **Resultado de este punto**

Al cerrar este notebook, tenemos:
- una definición clara de éxito,
- una noción explícita de fallo,
- y un marco de decisión **previo a cualquier métrica**.

Esto es la base sobre la que todo el resto del proyecto se construye.

---
---
---

## 8. Salida de este notebook

Este primer notebook no produce modelos, métricas ni gráficos.  
Produce algo más importante: **restricciones claras para todo lo que vendrá después**.

---

### 8.1 Resumen de la decisión definida

Hemos definido:
- qué decisión el sistema apoyará,
- en qué momento ocurre,
- y qué acción se ejecuta a partir de ella.

La decisión queda formalizada **antes de cualquier modelado**.

### 8.2 Resumen de los riesgos identificados

Se han identificado riesgos:
- operativos,
- financieros,
- y reputacionales,

asociados tanto al error como a la automatización indebida.

Estos riesgos **no desaparecen con métricas altas**.

### 8.3 Restricciones impuestas al modelo futuro

A partir de este análisis, el modelo futuro deberá:
- justificar cuándo decide,
- reconocer cuándo no debe hacerlo,
- y alinearse con los costos reales del error.

No se aceptará un modelo que solo optimice una métrica.

### 8.4 Cómo este criterio guiará todo el proyecto

Todo el proyecto se guiará por este principio:

> **El modelo es un medio.  
> La decisión es el fin.  
> El riesgo es el marco.**

Los notebooks siguientes construirán sobre esta base,  
introduciendo datos, modelos y métricas **sin perder de vista este criterio inicial**.

📌 **Cierre**

Este notebook establece el contrato conceptual del sistema.  
Nada de lo que siga puede violarlo.

---
---
---

## 9. Transición al siguiente notebook

Con la decisión, el riesgo y los criterios de éxito claramente definidos, el siguiente paso ya no es modelar, sino **mirar los datos con otros ojos**.

---

### 9.1 Qué se analizará a continuación

En el siguiente notebook se analizará:
- qué datos existen realmente,
- qué representan y qué no,
- y qué limitaciones imponen sobre la decisión definida.

No se buscará todavía entrenar modelos, sino **entender la realidad que los datos reflejan**.

### 9.2 Por qué ahora sí toca hablar de datos

Solo ahora tiene sentido introducir datos, porque:
- ya sabemos qué decisión deben apoyar,
- qué errores son críticos,
- y qué riesgos no se pueden ignorar.

Los datos dejarán de ser un CSV  
y pasarán a ser **una aproximación imperfecta a un problema real**.

### 9.3 Conexión con `02_data_reality_and_bias.ipynb`

El próximo notebook, `02_data_reality_and_bias.ipynb`, se centrará en:
- explorar la distribución real de los datos,
- detectar sesgos y desbalances,
- y evaluar si la data disponible permite —o no— apoyar la decisión definida.

📌 **Continuidad**

> Definir la decisión sin datos evita modelos inútiles.  
> Analizar los datos sin definir la decisión genera modelos peligrosos.

El siguiente notebook conecta ambos mundos.

---
---
---

📌 **Mensaje central del notebook**

> Un modelo puede ser técnicamente correcto y operativamente peligroso.
> Aquí definimos las reglas del juego antes de entrenar nada.